In [1]:
import pathlib

import cellpose
import numpy as np
import omnipose
import pandas as pd
import stardist
import tifffile
import torch
from cellpose_omni import core, io, transforms
from omnipose.utils import normalize99

2024-12-04 13:52:11,493	[INFO ]	[io.py      60   logger_setup      ]	WRITING LOG OUTPUT TO /home/lippincm/.cellpose/run.log


In [2]:
# check GPUs
print("Torch version: ", torch.__version__)
print("CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)
print("Number of CUDA devices: ", torch.cuda.device_count())
use_GPU = core.use_gpu()
print(">>> GPU activated? %d" % use_GPU)

Torch version:  2.5.1+cu124
CUDA available:  True
CUDA version:  12.4
Number of CUDA devices:  1
2024-12-04 13:52:11,777	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
>>> GPU activated? 1


In [3]:
# set path to the test image
image_file_path = pathlib.Path("../../data/z-stack_images/C3-2/C3-2_405.tif").resolve(
    strict=True
)

nuclei_z_stack = tifffile.imread(image_file_path)
print(nuclei_z_stack.shape)
# make a list of the z-stack images
nuclei_z_stack_list = [nuclei_z_stack[i] for i in range(nuclei_z_stack.shape[0])]
for i in nuclei_z_stack_list:
    print(i.shape)

(17, 1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)
(1537, 1540)


In [4]:
from cellpose_omni import core, models

model_name = "nuclei"

use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

2024-12-04 13:52:16,712	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
2024-12-04 13:52:16,713	[INFO ]	[models.py  427  __init__          ]	>>nuclei<< model set to be used
2024-12-04 13:52:16,713	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
2024-12-04 13:52:16,714	[INFO ]	[core.py    85   assign_device     ]	>>>> using GPU


resnet_torch.py (295): You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [5]:
model.eval(
    x=nuclei_z_stack_list,
)

  0%|          | 0/17 [00:00<?, ?it/s]

2024-12-04 13:53:31,891	[WARNING]	[models.py  699  eval              ]	input images must be a list of images, array of images, or dataloader
2024-12-04 13:53:32,082	[INFO ]	[utils.py   251  make_tiles_ND     ]	Running on tiles. Now normalizing each tile separately.


resnet_torch.py (295): You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


3 2 False False
2024-12-04 13:53:33,239	[WARNING]	[models.py  706  eval              ]	input images do not match the expected number of dimensions (2) and channels (2) of model.
2024-12-04 13:53:33,396	[INFO ]	[utils.py   251  make_tiles_ND     ]	Running on tiles. Now normalizing each tile separately.
3 2 False False
2024-12-04 13:53:34,354	[WARNING]	[models.py  706  eval              ]	input images do not match the expected number of dimensions (2) and channels (2) of model.
2024-12-04 13:53:34,476	[INFO ]	[utils.py   251  make_tiles_ND     ]	Running on tiles. Now normalizing each tile separately.
3 2 False False
2024-12-04 13:53:35,684	[WARNING]	[models.py  706  eval              ]	input images do not match the expected number of dimensions (2) and channels (2) of model.
2024-12-04 13:53:35,894	[INFO ]	[utils.py   251  make_tiles_ND     ]	Running on tiles. Now normalizing each tile separately.
3 2 False False
2024-12-04 13:53:37,111	[WARNING]	[models.py  706  eval              ]	inpu

([array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
